In [1]:
import pandas as pd
import numpy as np
import json
import pickle
import matplotlib.pyplot as plt
import logging
import scipy.stats as sts
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
import os

from arch import arch_model

In [2]:
ini_work_dir = os.getcwd()
os.chdir('..')

In [3]:
from rules_related import combine_rules_outputs
from whole_algorithm import train_model
from auxiliary import calc_log_returns, unpack_1d_parameters
from multivariate_normal_distribution import LongMultivariateNormal
from model import FuzzyVolatilityModel
from antecedent_fitting import fit_antecedent_params
from local_models import calculate_retraining_garch_forecasts

In [4]:
with open('config.json') as file:
    config = json.load(file)

In [5]:
files_folder = config['files_folders']['processed']
RESEARCH_RESULTS_FOLDER = config['files_folders']['research_results']
antecedent_params_folder = config['files_folders']['antecedent_params_sets']
logs_folder = config['files_folders']['logs']

In [6]:
TESTED_MODEL_FILE_NAME = 'tested_model_seasonal'
TESTED_MODEL_WOS_FILE_NAME = 'tested_model_wo_seas'

In [7]:
logger = logging.getLogger(f'test_calc')  # apf == antecedent params fitting

_log_file = f'{logs_folder}/{logger.name}.log'
logging.basicConfig(level=logging.INFO,
                    filename=_log_file,
                    filemode='w',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S')

In [8]:
pd.options.display.precision = 15
pd.options.display.max_colwidth = 300

# Reading

In [9]:
_files = ['fitting_consequent_params_seasonal_result_nb_0_2022-09-25 19-55-30.854710.pkl',
          'fitting_consequent_params_seasonal_result_nb_1_2022-09-25 20-16-50.717650.pkl']
res_big = [pd.read_pickle(f'{RESEARCH_RESULTS_FOLDER}/{_file}') for _file in _files]
_res = [_elem for res_piece in res_big for _elem in res_piece['result']].copy()

res = pd.DataFrame.from_records(_res).copy()

In [10]:
if (res['status'] != 0).any():
    raise Error('Not all models were fitted')

In [12]:
fitted_ap = res[res['rmse'] == res['rmse'].min()].iloc[0]['antecedent_params'].copy()
fitted_ap

[{'a': [1.0, 1.0],
  'b': [5.0, 1.0],
  'c': [5.0, 1.0],
  'd': [5.0, 5.0],
  'n_clusters': 2},
 {'centers': (-7.5, -1.5, 3.5, 4.0),
  'variances': (6, 4, 6, 6),
  'n_clusters': 4}]

In [20]:
res[res['rmse'] == res['rmse'].min()].shape

(20, 8)

In [21]:
fvm = res[res['rmse'] == res['rmse'].min()].iloc[0]['fvm']

In [22]:
test = res_big[0]['test'].copy()
data_to_cluster_test = res_big[0]['data_to_cluster_test'].copy()

# Live feeding

In [ ]:
%%time

fvm.feed_daily_data(test, data_to_cluster=data_to_cluster_test)

In [ ]:
_cur_time = str(pd.Timestamp.today()).replace(':', '-')
print(_cur_time)

with open(f'{RESEARCH_RESULTS_FOLDER}/{TESTED_MODEL_FILE_NAME}_{_cur_time}.pkl', 'wb') as _file:
    pickle.dump(fvm, _file)

In [ ]:
fvm.hist_output

In [ ]:
fvm.membership_degrees_hist

In [ ]:
_.shape

In [ ]:
pd.DataFrame.from_records(fvm._membership_degrees_hist)

In [ ]:
test.index.dayofweek

In [ ]:
res_big[0]['data_to_cluster_test'][0]

In [ ]:
fitted_ap

In [ ]:
fvm._parameters_hist

In [ ]:
fvm.show_ls_results()

In [ ]:
res_big[0].keys()

In [ ]:
mse = mean_squared_error(fvm.hist_output, test ** 2, squared=True)
rmse = mean_squared_error(fvm.hist_output, test ** 2, squared=False)

In [ ]:
rmse

In [ ]:
mse

In [ ]:
test ** 2

In [ ]:
fvm.hist_output

In [ ]:
fvm.hist_output.plot(grid=True, label='$h_t$')
(test ** 2).plot(grid=True, label='$y_t^2$')

plt.legend()

In [ ]:
test.plot(grid=True)